In [1]:
from qllm.models import llama
import torch 
def load_llama_7b():
    ava_model_sizes = llama.get_available_models()
    print(ava_model_sizes)
    llama_7b, tokenizer = llama.get_empty_model('7b')
    print("model_loaded")
    return llama_7b, tokenizer
llama_7b, tokenizer = load_llama_7b()

dict_keys(['7b', '13b', '33b', '65b'])
model_loaded


In [2]:
llama_7b.config.use_cache = False # not use cache to store the pevious computed result
# model structures
layers = llama_7b.model.layers
embed_tokens = llama_7b.model.embed_tokens
norm = llama_7b.model.norm
model_hidden_size = llama_7b.config.hidden_size
# the model dtype
dtype = next(iter(llama_7b.parameters())).dtype 

In [7]:
# create samples
seq_length = 2048 
nsamples = 2

In [18]:
dev="cuda:1"

In [19]:
inps = torch.ones(
        (nsamples, seq_length, model_hidden_size), dtype=dtype, device=dev
)
cache = {'i': 0, 'attention_mask': None} # manually set cache
outs = torch.zeros_like(inps)
attention_mask = cache['attention_mask']

In [20]:
# We only test sequential execution but not others
layer_length = len(layers)
for idx in range(layer_length):
    layer = layers[idx]
    layer = layer.cuda()
    outs = layer(inps, attention_mask=attention_mask)[0]
    layers[idx] = layer.cpu()
    del layer
    torch.cuda.empty_cache()
    inps, outs = outs, inps
    print(idx)

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 39.44 GiB total capacity; 38.64 GiB already allocated; 43.44 MiB free; 38.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
outs

In [10]:
# manually set cache
class Catcher(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module
    def forward(self, inp, **kwargs):
        inps[cache['i']] = inp
        cache['i'] += 1
        cache['attention_mask'] = kwargs['attention_mask']
        raise ValueError

In [11]:
dir(qllm)

['QLLM',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'models',
 'qllm_model']

In [9]:
from transformers import BloomConfig

In [10]:
AutoTokenizer(BloomConfig())

TypeError: AutoTokenizer.__init__() takes 1 positional argument but 2 were given

In [ ]:
dev = 'cuda:0'

In [ ]:
inps = torch.zeros(
        (args.nsamples, model.seqlen, model.config.hidden_size), dtype=dtype, device=dev
    )

In [7]:
embed_tokens

Embedding(32000, 4096, padding_idx=0)

In [4]:
layers

ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (act_fn): SiLUActivation()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)